# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = "os.getenv('OPENAI_API_BASE')"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0, openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

First, we load up our documents from the `data` directory:

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [ ]:
!pip install faiss-cpu==1.10.0


In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [ ]:
chat_history = []
query = "Can you get me a brief of AMAZING ANDAMAN?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

We can use this to easy implement chat conversations:

In [ ]:
chat_history = []

query = "In Paris package what is included and what is excluded ?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

In [ ]:
chat_history = [(query, result["answer"])]
query = "Give me quick brief of 6 Nights / 7 Days Swiss package ?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])